In [57]:
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as doc
import plotly.express as px
from dash.dependencies import Input,Output
import pandas as pd     #(version 1.0.0)
import numpy as np
import plotly           #(version 4.5.0)
import plotly.express as px
import numpy as numpy
import jupyter_dash             #(version 1.8.0)
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.io as pio
import plotly.graph_objects as go
import matplotlib.pyplot as plt
pio.templates.default = "plotly_dark"

In [58]:
circuits=pd.read_csv('datasets/circuits.csv')
constructor_results=pd.read_csv('datasets/constructor_results.csv')
constructor_standings=pd.read_csv('datasets/constructor_standings.csv')
constructors=pd.read_csv('datasets/constructors.csv')
driver_standings=pd.read_csv('datasets/driver_standings.csv')
drivers=pd.read_csv('datasets/drivers.csv')
lap_times=pd.read_csv('datasets/lap_times.csv')
pit_stops=pd.read_csv('datasets/pit_stops.csv')
qualifying=pd.read_csv('datasets/qualifying.csv')
races=pd.read_csv('datasets/races.csv')
results=pd.read_csv('datasets/results.csv')
seasons=pd.read_csv('datasets/seasons.csv')
status=pd.read_csv('datasets/status.csv')

## Laptime , position , delta and quali delta

In [59]:
turbohybrid=races.loc[races['year']>=2014]
turbohybrid=turbohybrid.merge(results,left_on='raceId',right_on='raceId')
turbohybrid=turbohybrid.merge(drivers,left_on='driverId',right_on='driverId')
turbohybrid=turbohybrid.merge(constructors,left_on='constructorId',right_on='constructorId')
turbohybridwinner=turbohybrid.loc[turbohybrid['position']=='1']
turbohybridwinner=turbohybridwinner.sort_values(by='raceId')
turbohybridwinner.drop(columns=['date', 'time_x',
       'url_x','number_x','url_y','url'],inplace=True)
racelaptimes=races.merge(lap_times,left_on='raceId',right_on='raceId')
racelaptimes=racelaptimes.loc[racelaptimes['year']>=2014]
racelaptimes=racelaptimes.merge(drivers,left_on='driverId',right_on='driverId')
racelaptimes['min_time_lap']=racelaptimes['milliseconds']
racelaptimes_temp=racelaptimes.merge(racelaptimes.groupby(['raceId','lap']).aggregate({'min_time_lap':'min'}),left_on=['raceId','lap'],right_on=['raceId','lap'])
racelaptimes_temp.drop(columns=['min_time_lap_x'],inplace=True)
racelaptimes_temp.rename(columns={'min_time_lap_y':'minlaptime'},inplace=True)
racelaptimes_temp['lapdelta']=racelaptimes_temp['milliseconds']-racelaptimes_temp['minlaptime']
racelaptimes_temp['lapdeltaseconds']=racelaptimes_temp['lapdelta']*0.001
racelaptimes_temp['milliseconds']=racelaptimes_temp['milliseconds']*0.001

In [4]:
racelaptimes_temp.head()

,raceId,year,round,circuitId,name,date,time_x,url_x,driverId,lap,...,number,code,forename,surname,dob,nationality,url_y,minlaptime,lapdelta,lapdeltaseconds
0,900,2014,1,1,Australian Grand Prix,2014-03-16,06:00:00,https://en.wikipedia.org/wiki/2014_Australian_...,3,1,...,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,102038,0,0.000
1,900,2014,1,1,Australian Grand Prix,2014-03-16,06:00:00,https://en.wikipedia.org/wiki/2014_Australian_...,817,1,...,3,RIC,Daniel,Ricciardo,1989-07-01,Australian,http://en.wikipedia.org/wiki/Daniel_Ricciardo,102038,1511,1.511
2,900,2014,1,1,Australian Grand Prix,2014-03-16,06:00:00,https://en.wikipedia.org/wiki/2014_Australian_...,825,1,...,20,MAG,Kevin,Magnussen,1992-10-05,Danish,http://en.wikipedia.org/wiki/Kevin_Magnussen,102038,3364,3.364
3,900,2014,1,1,Australian Grand Prix,2014-03-16,06:00:00,https://en.wikipedia.org/wiki/2014_Australian_...,807,1,...,27,HUL,Nico,Hülkenberg,1987-08-19,German,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,102038,4948,4.948
4,900,2014,1,1,Australian Grand Prix,2014-03-16,06:00:00,https://en.wikipedia.org/wiki/2014_Australian_...,4,1,...,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,102038,6228,6.228


In [61]:
year = 2018
name = 'Australian Grand Prix'

In [60]:
import pandas as pd     
import plotly           
import plotly.express as px
import dash_bootstrap_components as dbc  
import jupyter_dash             #(version 1.8.0)
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
df=racelaptimes_temp
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [69]:
card_1 =  dbc.Card([
    html.Div([   
    html.Div([

        html.Br(),
        html.Label(['Choose year and grandprix:'],style={'font-weight': 'bold', "text-align": "center",'margin':"15px" }),
        dcc.Dropdown(id='year_one',
            options=[{'label':x, 'value':x} for x in df.sort_values('year')['year'].unique()],
            value=2018,
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose year...',
            className='form-dropdown',
            style={'width':"100%",'margin':"15px",'color':'black'},
            persistence='string',
            persistence_type='memory'),
               
        dcc.Dropdown(id='gp name',
            options=[{'label':x, 'value':x} for x in df.sort_values('name')['name'].unique()],
            value='Australian Grand Prix',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose gp name...',
            className='form-dropdown',
            style={'width':"95%",'margin':"15px",'color':'black'},
            persistence='string',
            persistence_type='memory'),        
    ],className='two columns',style=dict(display='flex')),

])],color="dark",   # https://bootswatch.com/default/ for more card colors
    inverse=True,   # change color of text (black or white)
    outline=False, )

In [45]:
laptime = dbc.Card([
    html.Div([
        dcc.Graph(id='lap_graph')
    ],className='laptimes'),
])

In [46]:
lapdelta = dbc.Card([
    html.Div([
        dcc.Graph(id='lapdelta_graph')
    ],className='lapdeltas'),
])

In [70]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
   dbc.Col([dbc.Col(card_1),dbc.Col(laptime),dbc.Col(lapdelta)
            ])
])
@app.callback(
    Output('lap_graph','figure'), 
    Output('lapdelta_graph','figure'),
    [Input('year_one','value'),
    Input('gp name','value'),]
)

def build_graph(first_year,name):
    dff=df.loc[(df['year']==first_year)&(df['name']==name)]
    dff=dff.loc[dff['milliseconds']<200000]    
    fig=px.scatter(dff,x='lap',y='milliseconds',color='surname')
    fig1=px.box()
    fig1.add_trace(go.Box(x=dff['lap'],y=dff['lapdeltaseconds'],boxpoints=False))
    fig.update_layout()
    return fig,fig1


if __name__ == '__main__':
    app.run_server( port = 8010, threaded=True)

Dash app running on http://127.0.0.1:8010/
